# Isochrones for public parking lots in Tel Aviv

Created by [Dan Fishgold](https://dan.city)

Based on [Elad's LRT network isochrones](https://github.com/elad661/metroTLV_walkshed)

In [ ]:
# Basic setup
import json
import pandas as pd
from tqdm.notebook import tqdm
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
import shapely.geometry
import sklearn
import re
from fetch_statuses import get_lot_names

ox.settings.log_console = True

In [ ]:
# configure basic parameters
 # (Download the entire metropolitian area, and then some. Too bad OSM doesn't have a relationship for the Tel Aviv Metropolitan Area)
place = ["Tel Aviv District, Israel", "Center District, Israel"]
network_type = "walk"
trip_distances = [500]  # in meters

## Download and prep the street network

In [ ]:
# download the street network
graph = ox.graph_from_place(place, network_type=network_type)

In [ ]:
projected_graph = ox.project_graph(graph)
crs = projected_graph.graph['crs']

## Load night lots

From [Tel Aviv's GIS server](https://gisn.tel-aviv.gov.il/arcgis/rest/services/IView2/MapServer/488)

In [ ]:
raw_night_lots = gpd.read_file('https://gisn.tel-aviv.gov.il/arcgis/rest/services/IView2/MapServer/488/query?where=1%3D1&outFields=*&f=json').to_crs(crs).set_index('oid')

In [ ]:
night_lots = raw_night_lots
close_to_home = 'חניון קורב לבית - חניה חינם בלילות ובסופי שבוע לבעלות ולבעלי תו חניה מתאים, הנחה ל50% או 75% לתושבות ותושבי העיר'
paid = 'חניון בתשלום - הנחה של 50% או 75% לתשובות ותושבי העיר'
night_lots['lot_type'] = night_lots.sug_chenyon.astype('category').map({close_to_home: 'close to home', paid: 'paid'})
night_lots = night_lots[night_lots.lot_type.notna()].reindex(['shem', 'lot_type', 'geometry', 'ktovet', 'num_vehicles'], axis='columns')

## Load Ahuzot Hahof lots

In [ ]:
try:
    raw_ahuzot_lots = gpd.read_file("https://www.ahuzot.co.il/map/ParkingMap.aspx?gx=1234", driver='KML').to_crs(crs)
except:
    gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
    raw_ahuzot_lots = gpd.read_file("https://www.ahuzot.co.il/map/ParkingMap.aspx?gx=1234", driver='KML').to_crs(crs)

In [ ]:
ahuzot_lots = raw_ahuzot_lots
ahuzot_lots['id'] = ahuzot_lots.apply(lambda lot: int(re.search(r'href="https://www.ahuzot.co.il/Parking/ParkingDetails/\?ID=(\d+)"', lot.Description).group(1)), axis=1)
ahuzot_lots = ahuzot_lots.set_index('id')
del ahuzot_lots['Description']

In [ ]:
names = get_lot_names()
names_with_int_index = {int(id): name for (id, name) in names.items()}
ahuzot_lots['name'] = pd.Series(names_with_int_index)
del ahuzot_lots['Name']

# Match Ahuzot Hahof lots <> Night lots

I used a spatial join to match most of these, and then manually paired the rest

In [ ]:
pd.set_option('display.max_rows', 500)
joint_lots = night_lots.sjoin_nearest(ahuzot_lots, distance_col='distance', how='left', max_distance=65).reindex(['shem', 'name', 'distance', 'geometry','index_right', 'ktovet'], axis='columns').sort_values(by='distance', ascending=False)
# joint_lots

In [ ]:
lot_id_mapping = pd.Series({
    79: 114,
    1: 3,
    120: 135,
    74: 131,
    75: 38,
    10: 1,
    20: 94,
    21: 39,
    46: 57,
    14: 37,
    22: 26,
    16: 34,
    6: 7,
    34: 41,
    27: 44,
    7: 4,
    28: 18,
    23: 32,
    17: 19,
    31: 40,
    25: 33,
    18: 20,
    12: 31,
    5: 8,
    19: 53,
    67: 25,
    73: 127,
    68: 120,
    35: 42,
    33: 98,
    8: 48,
    3: 16,
    2: 45,
    9: 50,
    13: 15,
    60: 46,
    30: 29,
    69: 123,
    127: 13,
    121: 132,
    15: 21,
    39: 93,
    11: 28,
    24: 24,
    26: 23,
    4: 10,
    32: 99,
    29: 129,
    76: 134,
    77: 124,
    124: None,
    125: None,
})
night_lots['ahuzot_id'] = lot_id_mapping

lots = night_lots.join(ahuzot_lots.reindex(['name'], axis='columns'), on='ahuzot_id').reindex(['ahuzot_id', 'shem', 'name', 'ktovet', 'lot_type', 'geometry', 'num_vehicles'], axis='columns').rename(columns={'shem': 'gis_name', 'name': 'ahuzot_name', 'ktovet': 'address', 'num_vehicles': 'lot_capacity'})
lots

In [ ]:
lots['nearest_node'] = lots.apply(lambda lot: ox.distance.nearest_nodes(projected_graph, lot.geometry.x, lot.geometry.y), axis=1)

## Generate the isochrones

In [ ]:
# This function makes the isochrones, will be reused later
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = {}
    for trip_distance in sorted(trip_distances, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_distance, distance="length")

        node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index("id")

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union

        # try to fill in surrounded areas so shapes will appear solid and
        # blocks without white space inside them
        if infill and hasattr(new_iso, 'exterior'):
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys[trip_distance] = new_iso
    return isochrone_polys

def get_geojson_geometry(polygon):
    """Get geojson-compatible geometry, projected to a useful CRS"""
    geometry = ox.projection.project_geometry(polygon, crs=projected_graph.graph['crs'], to_latlong=True)[0]
    rounded_geometry = shapely.wkt.loads(shapely.wkt.dumps(geometry, rounding_precision=7))
    return shapely.geometry.mapping(rounded_geometry)



In [ ]:
distance_polys = {distance: [] for distance in trip_distances}
for lot_id, lot in tqdm(lots.iterrows(), total=lots.shape[0]):
    polys = make_iso_polys(projected_graph, lot['nearest_node'], edge_buff=25, node_buff=0, infill=True)
    for distance, polygon in polys.items():
        properties = lot.replace({pd.NA: None}).to_dict()
        lot_coordinates = ox.projection.project_geometry(properties['geometry'], crs=crs, to_latlong=True)[0]
        properties['gis_id'] = lot_id
        properties['lot_longitude'] = lot_coordinates.x
        properties['lot_latitude'] = lot_coordinates.y
        del properties['geometry']
        distance_polys[distance].append(dict(poly=polygon.simplify(1), properties=properties))

## Save the isochrones

In [ ]:
for distance, polys in distance_polys.items():
    features = []
    for polygon in polys:
        properties = {'distance': distance, **polygon['properties']}
        geometry = get_geojson_geometry(polygon['poly'])
        features.append(dict(type='Feature', properties=properties, geometry=geometry))

    # Save geojson
    geojson = { "type": "FeatureCollection", "features": features }
    with open(f'./parking_lot_isochrones_{distance}m.geojson', 'w') as f:
        json.dump(geojson, f, ensure_ascii=False)